In [1]:
from gurobipy import *
import snap
from graphviz import Digraph
import pandas as pd
import numpy as np

In [2]:
# G1 = snap.TUNGraph.New()
# G1.AddNode(0)
# G1.AddNode(1)
# G1.AddNode(2)
# G1.AddNode(3)
# G1.AddNode(4)
# G1.AddNode(5)
# G1.AddNode(6)
# G1.AddNode(7)
# G1.AddNode(8)
# G1.AddNode(9)
# G1.AddNode(10)
# G1.AddNode(11)
# G1.AddNode(12)
# G1.AddNode(13)
# G1.AddNode(14)
# G1.AddEdge(0,1)
# G1.AddEdge(0,2)
# G1.AddEdge(2,3)
# G1.AddEdge(1,3)
# G1.AddEdge(3,4)
# G1.AddEdge(4,5)
# G1.AddEdge(4,6)
# G1.AddEdge(0,7)
# G1.AddEdge(7,8)
# G1.AddEdge(7,9)
# G1.AddEdge(9,10)
# G1.AddEdge(0,11)
# G1.AddEdge(11,12)
# G1.AddEdge(11,13)
# G1.AddEdge(13,14)
# G1

In [3]:
# G1 = snap.GenRndGnm(snap.TUNGraph, 255, 600)
G1 = snap.LoadEdgeList(snap.TUNGraph, "graph.txt", 0, 1)

In [4]:
#參數設定
Budget = 2
init_S = 0

In [5]:
#刪除孤單的點
lonely_node = []
for NI in G1.Nodes():
    if NI.GetOutDeg()==0:
        lonely_node.append(NI.GetId())
        G1.DelNode(NI.GetId())

print("lonely_node = ",lonely_node)

lonely_node =  []


In [6]:
AllnodeID = []
InfectednodeID = [init_S]
V = [] #AllnodeID\0(s) #vulnerablenodeID
vaccinatednodeID = [] #vaccinated的node
for NI in G1.Nodes():
    AllnodeID.append(NI.GetId())
    if(NI.GetId()!=init_S):
        V.append(NI.GetId())

In [7]:
layer = 0
for NI in G1.Nodes():
    if(G1.GetShortPath(NI.GetId(),init_S)>layer):
        layer = G1.GetShortPath(NI.GetId(),init_S)
layer

6

In [8]:
#IP Programming

In [9]:
B = Budget #budget
S = init_S #初始感染點
#Create empty model
IP_model = Model('IP Vaccination Strategy')

#add variablies
#S = model.addVars([(u,t) for u in V for t in range(1,layer+1)],vtype=GRB.BINARY, name="S")
X = IP_model.addVars([(u,t) for u in V for t in range(1,layer+1) ],vtype=GRB.BINARY, name="X")
Y = IP_model.addVars([(v) for v in V],vtype=GRB.BINARY, name="Y")
IP_model.update()
#add objective function
# obj = 0
# for v in V:
#     obj = obj+Y[v]
IP_model.setObjective(quicksum(Y[v] for v in V) ,GRB.MAXIMIZE)

#constraint 1
IP_model.addConstrs(quicksum(X[u,t] for u in V)<=B for t in range(1,layer+1))
#constraint 2
#model.addConstrs(quicksum(X[u,t] for t in range(1,layer+1))<=1 for u in V) 
#constraint 3
IP_model.addConstrs(quicksum(X[u,t] for u in V for t in range(1,(G1.GetShortPath(S,v)-G1.GetShortPath(u,v))+1) if t>=0 )>= Y[v]  for v in V)
IP_model.update()
IP_model.optimize()

Academic license - for non-commercial use only - expires 2022-10-03
Using license file C:\Users\user\gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 372 rows, 2562 columns and 57717 nonzeros
Model fingerprint: 0x1573424e
Variable types: 0 continuous, 2562 integer (2562 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 4 rows and 962 columns
Presolve time: 0.11s
Presolved: 368 rows, 1600 columns, 55255 nonzeros
Variable types: 0 continuous, 1600 integer (1593 binary)

Root relaxation: objective 3.101453e+02, 741 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

In [10]:
print("IP Programming")
print("保護的個數=",IP_model.ObjVal)
for var in IP_model.getVars():
    if(var.x>0):
        print(var.varName,'\t', var.x)

IP Programming
保護的個數= 304.0
X[63,1] 	 1.0
X[118,1] 	 1.0
X[322,5] 	 1.0
X[325,2] 	 1.0
X[304,4] 	 1.0
X[282,3] 	 1.0
X[117,3] 	 1.0
X[183,5] 	 1.0
X[294,2] 	 1.0
X[242,4] 	 1.0
Y[63] 	 1.0
Y[118] 	 1.0
Y[1] 	 1.0
Y[328] 	 1.0
Y[364] 	 1.0
Y[2] 	 1.0
Y[83] 	 1.0
Y[90] 	 1.0
Y[123] 	 1.0
Y[167] 	 1.0
Y[248] 	 1.0
Y[322] 	 1.0
Y[368] 	 1.0
Y[3] 	 1.0
Y[124] 	 1.0
Y[208] 	 1.0
Y[266] 	 1.0
Y[268] 	 1.0
Y[4] 	 1.0
Y[161] 	 1.0
Y[209] 	 1.0
Y[247] 	 1.0
Y[60] 	 1.0
Y[7] 	 1.0
Y[59] 	 1.0
Y[200] 	 1.0
Y[219] 	 1.0
Y[273] 	 1.0
Y[8] 	 1.0
Y[88] 	 1.0
Y[158] 	 1.0
Y[323] 	 1.0
Y[330] 	 1.0
Y[9] 	 1.0
Y[75] 	 1.0
Y[191] 	 1.0
Y[245] 	 1.0
Y[325] 	 1.0
Y[43] 	 1.0
Y[11] 	 1.0
Y[49] 	 1.0
Y[12] 	 1.0
Y[286] 	 1.0
Y[46] 	 1.0
Y[76] 	 1.0
Y[116] 	 1.0
Y[256] 	 1.0
Y[14] 	 1.0
Y[72] 	 1.0
Y[81] 	 1.0
Y[313] 	 1.0
Y[347] 	 1.0
Y[355] 	 1.0
Y[15] 	 1.0
Y[139] 	 1.0
Y[170] 	 1.0
Y[288] 	 1.0
Y[16] 	 1.0
Y[82] 	 1.0
Y[106] 	 1.0
Y[92] 	 1.0
Y[252] 	 1.0
Y[304] 	 1.0
Y[18] 	 1.0
Y[127] 	 1.0
Y[223] 	 1.0


In [11]:
#LP Programming

In [12]:
B = Budget #budget
S = init_S #初始感染點
#Create empty model
LP_model = Model('LP Vaccination Strategy')

#add variablies
#S = model.addVars([(u,t) for u in V for t in range(1,layer+1)],vtype=GRB.BINARY, name="S")
X = LP_model.addVars([(u,t) for u in V for t in range(1,layer+1) ],lb=0,ub=1, name="X")
Y = LP_model.addVars([(v) for v in V],lb=0,ub=1, name="Y")
LP_model.update()
#add objective function
# obj = 0
# for v in V:
#     obj = obj+Y[v]
LP_model.setObjective(quicksum(Y[v] for v in V) ,GRB.MAXIMIZE)

#constraint 1
LP_model.addConstrs(quicksum(X[u,t] for u in V)<=B for t in range(1,layer+1))
#constraint 3
LP_model.addConstrs(quicksum(X[u,t] for u in V for t in range(1,(G1.GetShortPath(S,v)-G1.GetShortPath(u,v))+1) if t>=0 )>= Y[v]  for v in V)
LP_model.update()
LP_model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 372 rows, 2562 columns and 57717 nonzeros
Model fingerprint: 0xbeea4760
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 5 rows and 822 columns
Presolve time: 0.03s
Presolved: 367 rows, 1740 columns, 56868 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 6.335e+04
 Factor NZ  : 6.645e+04 (roughly 1 MByte of memory)
 Factor Ops : 1.606e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.01615934e+03  2.13084229e+03  8.38e+02 0.00e+00  3.07e+00     0s
   1   3.66646005e+02  1.4667

In [13]:
THE_X = [] #取出X[,]
X_Pro = [] #取出X[,]的機率
print("LP Programming")
print("保護的個數=",LP_model.ObjVal)
for var in LP_model.getVars():
    if(var.x>0):
        if(var.varName[0]=="X"):
            THE_X.append(var.varName) 
            X_Pro.append(var.x)
        print(var.varName,'\t', var.x)

LP Programming
保護的個數= 310.1453488372093
X[63,1] 	 0.6220930232558137
X[113,1] 	 0.10465116279069771
X[118,1] 	 0.4186046511627907
X[328,4] 	 0.052325581395350595
X[124,2] 	 0.16860465116279094
X[266,1] 	 0.2674418604651162
X[4,4] 	 0.5232558139534876
X[161,3] 	 0.029069767441861044
X[52,3] 	 0.16279069767441828
X[200,4] 	 0.09302325581395363
X[330,2] 	 0.005813953488371659
X[325,2] 	 0.3430232558139535
X[72,1] 	 0.15116279069767435
X[301,3] 	 0.12790697674418583
X[288,2] 	 0.017441860465116157
X[332,2] 	 0.01744186046511642
X[177,3] 	 0.18023255813953493
X[44,3] 	 0.1860465116279068
X[277,2] 	 0.24418604651162812
X[78,3] 	 0.07558139534883723
X[38,2] 	 0.02906976744186049
X[69,1] 	 0.20930232558139542
X[282,3] 	 0.4186046511627903
X[40,4] 	 0.2906976744186044
X[41,4] 	 0.06976744186046496
X[103,3] 	 0.16279069767441892
X[349,1] 	 0.029069767441860486
X[349,2] 	 0.16860465116279036
X[144,1] 	 0.09302325581395363
X[254,1] 	 0.029069767441860572
X[280,2] 	 0.08139534883720956
X[338,2] 	 0

In [14]:
#Threshold finding

In [15]:
def copy_graph(graph):
    tmpfile = '.copy.bin'

    # Saving to tmp file
    FOut = snap.TFOut(tmpfile)
    graph.Save(FOut)
    FOut.Flush()

    # Loading to new graph
    FIn = snap.TFIn(tmpfile)
    graphtype = type(graph)
    new_graph = graphtype.New()
    new_graph = new_graph.Load(FIn)

    return new_graph

In [16]:
import random
Threshold = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.99]
Final_threshold = 0
timer = 10
count = 0
maximum_saved_nodes = 0 #計算最多被保護的點
Final_strategy = [] #儲存最終最好的策略
print("Threshold Finding")
for TH in range(len(Threshold)):
    print("Threshold=",Threshold[TH])
    strategy_choose = [] #新策略(格式很亂)
    new_strategy = [[]for i in range(layer)]#新策略(格式清理過)

    for i in range(1,layer+1): #以每一個時間抓出來算機率
        XX = []
        XX_ori = []
        XX_Pro = []

        for x in range(len(THE_X)):
            if THE_X[x][-4]==",":
                if THE_X[x][-3:-1]==str(i):
                    XX.append(THE_X[x])
                    XX_Pro.append(X_Pro[x])
            elif THE_X[x][-2]==str(i):
                XX.append(THE_X[x])
                XX_Pro.append(X_Pro[x])
        if(len(XX)==0): #如果在某時沒有注射
            continue
        XX_ori = XX

        
        for i in range(len(XX_ori)): #判斷threshold #new Threshold
            if(XX_Pro[i]>Threshold[TH]):
                strategy_choose.append([XX[i]])
            
                
#     print("strategy",strategy_choose)#
    #清理格式    
    for i in range(len(strategy_choose)):
        if(strategy_choose[i][0][-4]==","): #注射時間是十位數
            new_strategy[int(strategy_choose[i][0][-3:-1])-1].append(strategy_choose[i][0][2:-4])
        else: #注射時間是個位數
            new_strategy[int(strategy_choose[i][0][-2])-1].append(strategy_choose[i][0][2:-3])

#     new_strategy = list(map(int, new_strategy)) #str-->int

    temp = "Y"
    for i in range(len(new_strategy)): #如果違反Budget
        if(len(new_strategy[i]) > B):
            temp = "N"
            break
    if(temp=="N"):
        continue
    print("選擇的策略=",new_strategy)
    ########################################################################開始注射策略
    G_temp = copy_graph(G1)
    S = init_S #初始感染點
    temp_vaccinated = []
    temp_infected = [S]
    temp_vulnerable = V.copy()    
    for i in range(len(new_strategy)): #開始依序注射
        if(len(new_strategy[i])==0):#如果這個時間點沒有注射
            a = temp_vulnerable.copy()
            for n in range(len(a)):#病毒開始擴散
                if(G_temp.GetShortPath(S,a[n])==i+1):
                    temp_infected.append(a[n])
                    temp_vulnerable.remove(a[n])
            continue
        TT_saved = []
        ttemp_saved = []
        for j in range(len(new_strategy[i])):
#             temp_saved = []
            b = temp_vulnerable.copy()
            if(int(new_strategy[i][j]) not in temp_vulnerable): #如果要注射的點已經被保護了(從圖中刪除)
#                 for n in range(len(b)):
#                      if(G_temp.GetShortPath(S,b[n])==i+1):#病毒開始擴散
#                         temp_infected.append(b[n])
#                         temp_vulnerable.remove(b[n])
                continue
                

            for k in range(len(temp_vulnerable)):#每一個點計算是否有被注射的點保護

                r  = i+1
                if(r<=(G_temp.GetShortPath(S,temp_vulnerable[k])-G_temp.GetShortPath(int(new_strategy[i][j]),temp_vulnerable[k]))):
                    ttemp_saved.append(temp_vulnerable[k])
        
        temp_saved = list(dict.fromkeys(ttemp_saved))
        for m in range(len(temp_saved)): #調整位置
            temp_vaccinated.append(temp_saved[m])
            temp_vulnerable.remove(temp_saved[m])
            G_temp.DelNode(temp_saved[m])

        a = temp_vulnerable.copy()
        for n in range(len(a)):#病毒開始擴散
            if(G_temp.GetShortPath(S,a[n])==i+1):
                temp_infected.append(a[n])
                temp_vulnerable.remove(a[n])

            
    print("保護個數=",len(temp_vaccinated))
    if(len(temp_vaccinated)>maximum_saved_nodes):
        maximum_saved_nodes = len(temp_vaccinated)
        Final_strategy = new_strategy
        Final_threshold = Threshold[TH]
    count = count+int(len(temp_vaccinated))
#     print("被保護的點=",temp_vaccinated)
# print("平均被保護個數=",count/timer)
print("Final_threshold = ",Final_threshold)
print("Final_strategy = ",Final_strategy)
print("Maximum saved node = ",maximum_saved_nodes)

Threshold Finding
Threshold= 0.1
Threshold= 0.2
Threshold= 0.3
Threshold= 0.4
選擇的策略= [['63', '118'], ['338'], ['282'], ['4', '97'], [], []]
保護個數= 273
Threshold= 0.5
選擇的策略= [['63'], [], [], ['4', '97'], [], []]
保護個數= 172
Threshold= 0.6
選擇的策略= [['63'], [], [], [], [], []]
保護個數= 170
Threshold= 0.7
選擇的策略= [[], [], [], [], [], []]
保護個數= 0
Threshold= 0.8
選擇的策略= [[], [], [], [], [], []]
保護個數= 0
Threshold= 0.9
選擇的策略= [[], [], [], [], [], []]
保護個數= 0
Threshold= 0.99
選擇的策略= [[], [], [], [], [], []]
保護個數= 0
Final_threshold =  0.4
Final_strategy =  [['63', '118'], ['338'], ['282'], ['4', '97'], [], []]
Maximum saved node =  273
